In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# step1 convert image to embedding vector sequence
#写法1 DNN视角
def image2emb_navie(image, patch_size, weight):
    #image shape :bs*channels*h*w 
    patch = F.unfold(image, kernel_size = patch_size, stride = patch_size)
    #print(patch.shape) #(bs,patch_depth,(image_h/patchsize)*(image_w/patch_size))
    patch = patch.transpose(-1, -2)
    patch_embdding = patch @ weight
    return patch_embdding


#test code for image2emb
patch_size = 4
bs, ic, image_h, image_w = 1,3,8,8
model_dim = 8
patch_depth = patch_size * patch_size * ic
image = torch.randn(bs, ic, image_h, image_w)
weight = torch.randn(patch_depth, model_dim)
patch_embedding_navie = image2emb_navie(image, patch_size,weight)
patch_embedding_navie.shape

torch.Size([1, 4, 8])

In [11]:
#写法2卷积视角，卷积步长等于patch长度
def image2emb_conv(image, kernel, stride):
    conv_output = F.conv2d(image, kernel, stride = stride)  #bs*oc*oh*ow
    bs, oc, oh, ow = conv_output.shape
    patch_embedding = conv_output.reshape((bs, oc, oh * ow)).transpose(-1, -2)
    return patch_embedding

#test code for image2emb
patch_size = 4
bs, ic, image_h, image_w = 1,3,8,8
model_dim = 8
patch_depth = patch_size * patch_size * ic
image = torch.randn(bs, ic, image_h, image_w)
weight = torch.randn(patch_depth, model_dim) #model_dim最后是输出通道数目，patch_depth是kernel.numel * 输入通道数目

kernel = weight.transpose(0, 1).reshape(-1, ic, patch_size, patch_size)# oc*ic*kh*kw
patch_embedding_conv = image2emb_conv(image, kernel,patch_size)
patch_embedding_conv
patch_embedding_navie = image2emb_navie(image, patch_size,weight)
flag = torch.allclose(patch_embedding_conv,patch_embedding_navie)
print(flag)

True


torch.tile和repeat都可以复制，可以看[博客](https://www.cnblogs.com/nxf-rabbit75/p/10055932.html)

In [29]:
#step2 序列开头增加一个cls
#注意一下: VisionTransformer中这里可学习的变量cls_token_embedding是一个长度为model_dim的向量，
# 与batchsize无关，concat的时候对batch维度复制一下即可。特此更正一下。
max_num_token = 16
cls_token_embedding = torch.randn(1, 1, model_dim, requires_grad=True)

cls_token_embedding.repeat(bs,1,1).shape
token_embedding = torch.cat([cls_token_embedding.repeat(bs,1,1),patch_embedding_conv],dim = 1)

# step3 增加位置编码，采用随机的可学习的embedding,各种位置编码性能差不多
position_embedding_table = torch.randn(max_num_token, model_dim, requires_grad=True)
seq_len = token_embedding.shape[1] #由于图中大小确定，mask也相对确定，不像nlp那种句子长度不确定
position_embdding = torch.tile(position_embedding_table[:seq_len], [token_embedding.shape[0],1,1])#batch复制
# position_embdding.shape
token_embedding += position_embdding

In [36]:
#step4. 将embedding送入encoder
encoder_layer = nn.TransformerEncoderLayer(d_model = model_dim, nhead = 8)
transformer_encoder = nn.TransformerEncoder(encoder_layer,num_layers=6)
encoder_output = transformer_encoder(token_embedding)

#step5 做分类
num_classes = 10
label = torch.randint(10,(bs,))
# print(label)
cls_token_output = encoder_output[:,0,:]
linear_layer = nn.Linear(model_dim, num_classes)
logits = linear_layer(cls_token_embedding)
logits = logits.squeeze(1) #要把中间的维度消除掉
print(logits.shape)
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, label)
print(loss)


torch.Size([1, 10])
tensor(3.1925, grad_fn=<NllLossBackward>)
